# 학습 관련 기술들     
---
### 이 단원에서 다룰 것
1. 가중치 매개변수의 최적값을 탐색하는 최적화 방법
2. 가중치 매개변수의 초깃값    
3. 하이퍼파라미터 설정 방법     
4. 오버피팅의 대응책인 가중치 감소와 드롭아웃 등의 정규화 방법     
5. 배치 정규화(New!)  
---

### 6.1 매개변수 갱신    

- 신경망 학습의 목적 : 손실 함수의 값을 가능한 한 낮추는 매개변수를 찾는 것.    
- == 매개변수의 최적값을 찾는 것, 즉 __최적화(optimization)__        
---
###### 최적화는 생각보다 어렵다. 왜?       
- 매개변수 공간은 너무 넓고 복잡하다.    
- 심층 신경망에서는 매개변수가 너무너무너ㅜㅜ무 많아져서 찾기가 더! 어렵다.     
---
- 지금까지는 최적화 방법으로 __'매개변수 기울기(미분)'__ 을 이용했다.     
- 이것은 확률적 경사 하강법(SGD) : 매개변수 기울기를 구해, 기울어진 방향으로 매개변수 값을 계속 갱신하는 방법    

### 6.3 확률적 경사 하강법(SGD)    


In [3]:
class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr
        
    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr * grad[key]

* lr == 학습률(인스턴스 변수로 유지 << 사람이 변경하지 않음)     
* 최적화를 수행할 클래스를 분리해 구현하면 모든 기능을 모듈화하기 좋다. (update(params, grad))

#### 6.3.1 SGD의 단점

$f(x, y) = 1/20*x^2 + y^2$   

- y축의 변화는 큰데 x축 변화는 매우 미미한 그래프

- 비등방성 함수(방향에 따라 성질, 기울기가 달라지는 함수)에서는 탐색 경로가 비효율적이다. (매번 위치에 의해 파라미터가 바뀌므로)    
- 더 영리하게 움직일 수 있는 방법은 없는지?     
- SGD가 지그재그로 움직이는 것은 SGD가 기울어진 방향이 본래의 최솟값과 다른 방향을 가리켜서라는 것을 상기해야 한다.       

---

> 이러한 단점을 개선해주는 모멘텀, AdaGrad, Adam 기법에 대해 알아보자!

### 6.1.4 모멘텀(momentum, '운동량')

In [4]:
class Momentum:
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        slef.v = None
        
    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.item():
                self.v[key] = np.zeros_like(val)
                
        for key in params.key():
            self.v[key] = self.momentum * self.v[key] - self.lr*grads[key]
            params[key] += self.v[key]

### 마치 큰 보울 안에서 움직이는 공의 물리법칙처럼

- 기울기를 이전에 움직였던 '방향'을 적용하여 갱신한다.    
- 단, 아무 움직임이 없는 곳에는 $av$ 라는 항을 넣어 물체가 아무런 힘을 받지 않을 때라도 서서히 하강하도록(움직이도록) 한다.  
- 부드러운 곡선 S자로 움직인다.

### 6.1.5 AdaGrad

* 신경망 학습에는 학습률이 매우 중요하다.    
* __학습률이 너무 작으면 학습시간이 너무 길어지고, 반대로 너무 크면 발산하여 학습이 제대로 이뤄지지 않는다.__    
---
* 이 학습률을 정하는 기술 : 학습률 감소(learning rate decay) - 학습률을 학습하면서 서서히 줄여나가는 방법     
* 처음에는 크게 학습하다가 점차 작게 학습하는 방법    
*  *h(기존 기울기 값을 제곱하여 계속 곱해준다)  매개변수를 갱신할 때는 $1/sqrt(h)$ 하여 학습률을 조정한다.     
- 이 때 갈수록 줄어드니 0이 될수도 있다는 단점을 보완하여, 옛날 기울기는 서서히 잊어가고 새 기울기에 중점을 많이 주는 RMSProp라는 방법도 있다.

In [5]:
class AdaGrad:
    def __init__(self, lr = 0.01):
        self.lr = lr
        self.h = None
        
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key, val in params.item():
                self.h[val] = np.zeros_like(val)
                
        for key in params.keys():
            self.h[key] += grads[key] * grads[key]
            params[key] -= self.lr * grads[key]  / np.sqrt(self.h[key]) +1e-7

- 처음에는 크게 움직이고, 갱신도 큰 폭(큰 값의 제곱분의 1)으로 작아지도록 조정된다.
- y축 방향으로 갱신 강도가 빠르게 약해지고, 지그재그 움직임이 줄어든다.

### 6.1.6 Adam

- 모멘텀은 공이 구르는 것 같은 움직임, AdaGrad는 매개변수의 원소마다 적응적으로 갱신속도를 조정.    
- 이를 섞어주면 Adam   

### 6.1.7 어느 갱신 방법을 이용할 것인가?    